In [2]:
import obspy
from obspy.clients.filesystem.sds import Client as sdsclient
from SAM import RSAM
from pathlib import Path
srcdir = '/data/Montserrat/LaharStudy'
SDS_DIR = '/data/SDS'
SAM_DIR = '/data/SAM'
startTime = obspy.core.UTCDateTime(2018,1,1)
endTime = obspy.core.UTCDateTime(2021,1,1)
network = 'MV'

In [ ]:
# Run through SDS archive in /data/Montserrat/LaharStudy, which has .miniseed appended to filename
import os
import glob
for yeardir in sorted(glob.glob(os.path.join(srcdir, '20??'))):
    networkdir = os.path.join(yeardir, network)
    for stationdir in sorted(glob.glob(os.path.join(networkdir, '*'))):
        for channeldir in sorted(glob.glob(os.path.join(stationdir, '*.[DL]'))):
            if 'SOH.L' in channeldir:
                channeldir = os.path.join(channeldir, 'soh')
            for filepath in sorted(glob.glob(os.path.join(channeldir, f'{network}*'))):
                if os.path.isfile(filepath):
                    newfilepath = filepath.replace(srcdir, SDS_DIR)
                    if '.miniseed' in filepath:
                        newfilepath = newfilepath.replace('.miniseed', '')
                    newdirpath = os.path.dirname(newfilepath)
                    if not os.path.isdir(newdirpath):
                        os.makedirs(newdirpath)
                    os.system(f"cp {filepath} {newfilepath}")


In [ ]:
# Compute RSAM in 1-day chunks for multiple network-station-location-channel's
mySDSclient = sdsclient(SDS_DIR, sds_type='D', format='MSEED', fileborder_seconds=30, fileborder_samples=5000)

In [ ]:
# Just some sanity checking that the SDSclient is connecting to data
print(mySDSclient)
print(mySDSclient.get_all_nslc()) # outputs a list of (net, sta, loc, chan) available
percent_available, num_gaps = mySDSclient.get_availability_percentage(network, 'ALEJ', '', 'HH*', startTime, endTime, sds_type='D')
print(percent_available, num_gaps) # looks like 92.3% availability for combined ALEJ channels from 2012-04-01 to 2012-07-01

[('NR', 'ALEJ', '', 'HHE'), ('NR', 'ALEJ', '', 'HHN'), ('NR', 'ALEJ', '', 'HHZ'), ('NR', 'ALF1', '', 'HHE'), ('NR', 'ALF1', '', 'HHN'), ('NR', 'ALF1', '', 'HHZ'), ('NR', 'ANIL', '', 'HHE'), ('NR', 'ANIL', '', 'HHN'), ('NR', 'ANIL', '', 'HHZ'), ('NR', 'ESME', '', 'HHE'), ('NR', 'ESME', '', 'HHN'), ('NR', 'ESME', '', 'HHZ'), ('NR', 'LAGU', '', 'HHE'), ('NR', 'LAGU', '', 'HHN'), ('NR', 'LAGU', '', 'HHZ'), ('NR', 'PIRA', '', 'HHE'), ('NR', 'PIRA', '', 'HHN'), ('NR', 'PIRA', '', 'HHZ'), ('NR', 'RREF', '', 'HHE'), ('NR', 'RREF', '', 'HHM'), ('NR', 'RREF', '', 'HHN'), ('NR', 'RREF', '', 'HHZ'), ('NR', 'RUB4', '', 'HHE'), ('NR', 'RUB4', '', 'HHN'), ('NR', 'RUB4', '', 'HHZ'), ('NR', 'TAPI', '', 'HHE'), ('NR', 'TAPI', '', 'HHN'), ('NR', 'TAPI', '', 'HHZ'), ('NR', 'TOCH', '', 'HHE'), ('NR', 'TOCH', '', 'HHN'), ('NR', 'TOCH', '', 'HHZ')]
0.922739493030118 10


In [ ]:
secondsPerDay = 60 * 60 * 24
numDays = (endTime-startTime)/secondsPerDay
daytime = startTime
while daytime < endTime:
    print(f'Loading Stream data for {daytime.strftime("%Y-%m-%d")}')
    if percent_available>0:
        st = mySDSclient.get_waveforms(network, "*", "*", "[SBEHCD]*", daytime, daytime+secondsPerDay)
        print(st)
        print(f'- got {len(st)} Trace ids')
        if len(st)>0:
            print(f'Computing RSAM metrics for {daytime}, and saving to pickle files')
            rsam24h = RSAM(stream=st, sampling_interval=60)
            rsam24h.write(SAM_DIR, ext='csv', overwrite=True)
    daytime += secondsPerDay

Loading Stream data for 2012-04-01
28 Trace(s) in Stream:

NR.ALEJ..HHE | 2012-04-01T00:00:00.000000Z - 2012-04-01T23:59:59.990000Z | 100.0 Hz, 8640000 samples
...
(26 other traces)
...
NR.TOCH..HHZ | 2012-04-01T00:00:00.000000Z - 2012-04-01T23:59:59.990000Z | 100.0 Hz, 8640000 samples

[Use "print(Stream.__str__(extended=True))" to print all Traces]
- got 28 Trace ids
Computing RSAM metrics for 2012-04-01T00:00:00.000000Z, and saving to pickle files
NR.RREF..HHM | 2012-04-01T00:00:00.000000Z - 2012-04-01T23:59:59.750000Z | 4.0 Hz, 345600 samples: bad sampling rate. Skipping.
write
/data/OVSM/SAM/RSAM_NR.ALEJ..HHE_2012_60s.csv
Writing /data/OVSM/SAM/RSAM_NR.ALEJ..HHE_2012_60s.csv
/data/OVSM/SAM/RSAM_NR.ALEJ..HHN_2012_60s.csv
Writing /data/OVSM/SAM/RSAM_NR.ALEJ..HHN_2012_60s.csv
/data/OVSM/SAM/RSAM_NR.ALEJ..HHZ_2012_60s.csv
Writing /data/OVSM/SAM/RSAM_NR.ALEJ..HHZ_2012_60s.csv
/data/OVSM/SAM/RSAM_NR.ALF1..HHE_2012_60s.csv
Writing /data/OVSM/SAM/RSAM_NR.ALF1..HHE_2012_60s.csv
/data/OVSM/